In [1]:
import pandas as pd
import numpy as np
import math
import yaml
from pathlib import Path

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from scripts.db_psql_model import DatabaseCursor
from scripts.tournament import Tournament

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]

MoM FFBL Weekly Rankings Replacement 

In [ ]:
"""
MoM FFBL Weekly Rankings Replacement 2pt system
"""
matchups_query = """SELECT * FROM dev.reg_season_matchups"""
weekly_points_query = """SELECT * FROM dev.weekly_team_pts"""
teams_query = """SELECT team_key, name, nickname, game_id FROM dev.league_teams"""
settings_query = """SELECT * FROM dev.league_settings"""
meta_query = """SELECT game_id, league_id, end_week FROM dev.league_metadata"""
matchups = (
    DatabaseCursor(PATH, option_schema="raw")
    .copy_data_from_postgres(matchups_query)
    .drop_duplicates()
)

weekly_points = (
    DatabaseCursor(PATH, option_schema="raw")
    .copy_data_from_postgres(weekly_points_query)
    .drop_duplicates()
)

weekly_points["team_key"] = (
    weekly_points["game_id"].astype(str)
    + ".l."
    + weekly_points["league_id"].astype(str)
    + ".t."
    + weekly_points["team_id"].astype(str)
)
weekly_points.drop_duplicates(subset=["team_key", "week"], inplace=True)

teams = (
    DatabaseCursor(PATH, option_schema="raw")
    .copy_data_from_postgres(teams_query)
    .drop_duplicates()
)

settings = (
    DatabaseCursor(PATH, option_schema="raw")
    .copy_data_from_postgres(settings_query)
    .drop_duplicates()
)

metadata = (
    DatabaseCursor(PATH, option_schema="raw")
    .copy_data_from_postgres(meta_query)
    .drop_duplicates()
)

settings = settings.merge(metadata, how="left", on=["game_id", "league_id"])

matchups_a = matchups.copy()
matchups_b = matchups.copy()

matchups_b_cols = list(matchups_b.columns)

rename_columns = {}
for col in matchups_b_cols:
    if "team_a" in col:
        rename_columns[col] = f"team_b{col[6:]}"
    elif "team_b" in col:
        rename_columns[col] = f"team_a{col[6:]}"

matchups_b.rename(columns=rename_columns, inplace=True)

matchups = pd.concat([matchups_a, matchups_b])

matchups.sort_values(["week_start", "team_a_team_key"], inplace=True)

matchups.reset_index(drop=True, inplace=True)

matchups.drop(
    [
        "is_matchup_recap_available",
        "is_tied",
        "matchup_recap_title",
        "matchup_recap_url",
        "status",
        "league_id",
        "team_a_grade",
        "team_b_grade",
    ],
    axis=1,
    inplace=True,
)

game_keys = list(matchups["game_id"].unique())

for key in game_keys:

    # logic to help create playoff brackets
    playoff_start_week = settings["playoff_start_week"][
        settings["game_id"] == key
    ].values[0]
    playoff_end_week = settings["end_week"][settings["game_id"] == key].values[0]
    playoff_weeks = range(playoff_start_week, playoff_end_week + 1)
    max_teams = settings["max_teams"][settings["game_id"] == key].values[0]
    num_playoff_teams = settings["num_playoff_teams"][
        settings["game_id"] == key
    ].values[0]
    num_conso_teams = settings["num_playoff_consolation_teams"][
        settings["game_id"] == key
    ].values[0]
    max_teams = settings["max_teams"][settings["game_id"] == key].values[0]
    num_toliet_teams = (
        (
            max_teams - num_playoff_teams
            if num_playoff_teams >= (max_teams / 2)
            else num_playoff_teams
        )
        if num_conso_teams != 0
        else False
    )

    # logic to create regular season board - matchups each week with running rankings
    one_reg_season = matchups[
        (matchups["game_id"] == key) & (matchups["week"] < playoff_start_week)
    ]
    one_reg_season["win_loss"] = np.where(
        one_reg_season["winner_team_key"] == one_reg_season["team_a_team_key"], "W", "L"
    )
    one_reg_season["pts_system_weekly_rank"] = one_reg_season.groupby(
        ["week", "game_id"]
    )["team_a_points"].rank("first", ascending=False)
    one_reg_season["pts_score"] = np.where(
        one_reg_season["pts_system_weekly_rank"] <= 5, 1, 0
    )
    one_reg_season = one_reg_season.merge(
        teams, how="left", left_on="team_a_team_key", right_on="team_key"
    )
    one_reg_season["team_a_name"] = one_reg_season["name"].fillna(
        one_reg_season["team_a_team_key"]
    )
    one_reg_season["team_a_nickname"] = one_reg_season["nickname"].fillna(
        one_reg_season["team_a_team_key"]
    )
    one_reg_season["game_id_a"] = one_reg_season["game_id_x"].fillna(
        one_reg_season["game_id_y"]
    )
    one_reg_season.drop(["name", "nickname"], axis=1, inplace=True)
    one_reg_season = one_reg_season.merge(
        teams, how="left", left_on="team_b_team_key", right_on="team_key"
    )
    one_reg_season["team_b_name"] = one_reg_season["name"].fillna(
        one_reg_season["team_b_team_key"]
    )
    one_reg_season["team_b_nickname"] = one_reg_season["nickname"].fillna(
        one_reg_season["team_b_team_key"]
    )
    one_reg_season.drop(["name", "nickname"], axis=1, inplace=True)
    one_reg_season["game_id"] = one_reg_season["game_id_a"]

    one_reg_season = one_reg_season.rename(
        columns={
            "team_a_team_key": "team_key",
            "team_a_name": "team_name",
            "team_a_nickname": "team_nickname",
            "team_a_points": "team_pts",
            "team_a_projected_points": "team_pro_pts",
            "team_b_team_key": "opp_team_key",
            "team_b_name": "opp_team_name",
            "team_b_nickname": "opp_team_nickname",
            "team_b_points": "opp_pts",
            "team_b_projected_points": "opp_pro_pts",
        }
    )

    one_reg_season = one_reg_season[
        [
            "game_id",
            "team_key",
            "team_name",
            "team_nickname",
            "week",
            "win_loss",
            "pts_score",
            "team_pts",
            "team_pro_pts",
            "opp_team_key",
            "opp_team_name",
            "opp_team_nickname",
            "opp_pts",
            "opp_pro_pts",
        ]
    ]

    one_reg_season["ttl_pts_for_run"] = one_reg_season.groupby(["team_key"])[
        "team_pts"
    ].cumsum()
    one_reg_season["pts_for_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_pts_for_run"]
        .rank(method="min", ascending=False)
        .astype(int)
    )

    one_reg_season["ttl_pro_pts_for_run"] = one_reg_season.groupby(["team_key"])[
        "team_pro_pts"
    ].cumsum()
    one_reg_season["pro_pts_for_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_pro_pts_for_run"]
        .rank(method="min", ascending=False)
        .astype(int)
    )

    one_reg_season["ttl_pts_agst_run"] = one_reg_season.groupby(["team_key"])[
        "opp_pts"
    ].cumsum()
    one_reg_season["pts_agst_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_pts_agst_run"]
        .rank(method="max", ascending=True)
        .astype(int)
    )

    one_reg_season["ttl_pro_pts_agst_run"] = one_reg_season.groupby(["team_key"])[
        "opp_pro_pts"
    ].cumsum()
    one_reg_season["pro_pts_agst_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_pro_pts_agst_run"]
        .rank(method="max", ascending=True)
        .astype(int)
    )

    one_reg_season["ttl_wins_run"] = (
        one_reg_season["win_loss"].eq("W").groupby(one_reg_season["team_key"]).cumsum()
    )
    one_reg_season["ttl_loss_run"] = (
        one_reg_season["win_loss"].eq("L").groupby(one_reg_season["team_key"]).cumsum()
    )
    one_reg_season["w_l_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_wins_run"]
        .rank(method="min", ascending=False)
        .astype(int)
    )

    one_reg_season["ttl_pts_score_run"] = one_reg_season.groupby(["team_key"])[
        "pts_score"
    ].cumsum()
    one_reg_season["ttl_2pt_score_run"] = (
        one_reg_season["ttl_pts_score_run"] + one_reg_season["ttl_wins_run"]
    )
    one_reg_season["2pt_score_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_2pt_score_run"]
        .rank(method="min", ascending=False)
        .astype(int)
    )

    if key >= 390:
        one_reg_season["tuple"] = one_reg_season[
            ["2pt_score_rank_run", "pts_for_rank_run"]
        ].apply(tuple, axis=1)
    else:
        one_reg_season["tuple"] = one_reg_season[
            ["w_l_rank_run", "pts_for_rank_run"]
        ].apply(tuple, axis=1)

    one_reg_season.insert(
        5,
        "reg_season_rank_run",
        one_reg_season.groupby(["week"])["tuple"]
        .rank(method="min", ascending=True)
        .astype(int),
    )

    one_reg_season.sort_values(
        ["week", "reg_season_rank_run"], ascending=[True, True], inplace=True
    )

    reg_season_rank = one_reg_season[
        ["game_id", "team_key", "team_name", "team_nickname", "reg_season_rank_run"]
    ].tail(max_teams)

    DatabaseCursor(PATH, option_schema="dev").copy_table_to_postgres_new(
        df=one_reg_season, table=f"reg_season_board_{str(key)}", first_time="YES"
    )

    # logic to create the playoff matchup board/bracket works for playoff, consolation, and toilet
    one_playoff_season = weekly_points[
        (weekly_points["game_id"] == key)
        & (weekly_points["week"] >= playoff_start_week)
    ]
    one_playoff_season = one_playoff_season.merge(
        teams, how="left", left_on="team_key", right_on="team_key"
    )
    one_playoff_season = one_playoff_season[
        [
            "game_id_x",
            "team_key",
            "name",
            "nickname",
            "week",
            "final_points",
            "projected_points",
        ]
    ]
    one_playoff_season = one_playoff_season.rename(
        columns={
            "game_id_x": "game_id",
            "name": "team_name",
            "nickname": "team_nickname",
            "final_points": "team_pts",
            "projected_points": "team_pro_pts",
        }
    )
    one_playoff_season.sort_values(["team_key", "week"], inplace=True)
    one_playoff_season["ttl_pts_for_run"] = one_playoff_season.groupby(["team_key"])[
        "team_pts"
    ].cumsum()
    one_playoff_season["ttl_pro_pts_for_run"] = one_playoff_season.groupby(
        ["team_key"]
    )["team_pro_pts"].cumsum()

    one_playoff_season = one_playoff_season.merge(
        reg_season_rank,
        how="left",
        on=["game_id", "team_key", "team_name", "team_nickname"],
    )

    one_playoff_season.rename(
        columns={"reg_season_rank_run": "reg_season_rank"}, inplace=True
    )

    one_playoff_season.sort_values(
        ["week", "reg_season_rank"], ascending=[True, True], inplace=True
    )

    playoff_teams = list(
        one_playoff_season["team_key"][
            one_playoff_season["reg_season_rank"] <= num_playoff_teams
        ].unique()
    )
    if num_toliet_teams:
        if key == 314:
            conso_teams = list(
                one_playoff_season["team_key"][
                    one_playoff_season["reg_season_rank"]
                    <= (max_teams - num_playoff_teams)
                ].unique()
            )
            conso_teams = [tm for tm in conso_teams if tm not in playoff_teams]
            toilet_teams = False

        elif key >= 406:
            conso_teams = list(
                one_playoff_season["team_key"][
                    one_playoff_season["reg_season_rank"] > num_playoff_teams
                ].unique()
            )
            toilet_teams = list(
                one_playoff_season["team_key"][
                    one_playoff_season["reg_season_rank"]
                    > (max_teams - num_conso_teams)
                ].unique()
            )
            conso_teams = [tm for tm in conso_teams if tm not in toilet_teams]

        else:
            conso_teams = False
            toilet_teams = list(
                one_playoff_season["team_key"][
                    one_playoff_season["reg_season_rank"]
                    > (max_teams - num_toliet_teams)
                ].unique()
            )
    else:
        toilet_teams = False
        conso_teams = False

    playoff_end_week_mask = one_playoff_season["week"] == playoff_end_week
    one_playoff_season.insert(1, "finish", np.nan)

    playoff_bracket = Tournament(playoff_teams)
    num_of_rounds = int(math.ceil(math.log2(len(playoff_teams))))
    for week in playoff_weeks:
        for match in playoff_bracket.get_active_matches():
            right_comp = match.get_participants()[0].get_competitor()
            right_score = one_playoff_season["team_pts"][
                (one_playoff_season["team_key"] == right_comp)
                & (one_playoff_season["week"] == week)
            ].values[0]
            right_pro_score = one_playoff_season["team_pro_pts"][
                (one_playoff_season["team_key"] == right_comp)
                & (one_playoff_season["week"] == week)
            ].values[0]
            left_comp = match.get_participants()[1].get_competitor()
            left_score = one_playoff_season["team_pts"][
                (one_playoff_season["team_key"] == left_comp)
                & (one_playoff_season["week"] == week)
            ].values[0]
            left_pro_score = one_playoff_season["team_pro_pts"][
                (one_playoff_season["team_key"] == left_comp)
                & (one_playoff_season["week"] == week)
            ].values[0]
            one_playoff_season.loc[
                (one_playoff_season["team_key"] == right_comp)
                & (one_playoff_season["week"] == week),
                "po_opp",
            ] = left_comp
            one_playoff_season.loc[
                (one_playoff_season["team_key"] == right_comp)
                & (one_playoff_season["week"] == week),
                "po_opp_pts",
            ] = left_score
            one_playoff_season.loc[
                (one_playoff_season["team_key"] == right_comp)
                & (one_playoff_season["week"] == week),
                "po_opp_pro_pts",
            ] = left_pro_score
            one_playoff_season.loc[
                (one_playoff_season["team_key"] == left_comp)
                & (one_playoff_season["week"] == week),
                "po_opp",
            ] = right_comp
            one_playoff_season.loc[
                (one_playoff_season["team_key"] == left_comp)
                & (one_playoff_season["week"] == week),
                "po_opp_pts",
            ] = right_score
            one_playoff_season.loc[
                (one_playoff_season["team_key"] == left_comp)
                & (one_playoff_season["week"] == week),
                "po_opp_pro_pts",
            ] = right_pro_score
            if right_score > left_score:
                match.set_winner(right_comp)
            elif right_score < left_score:
                match.set_winner(left_comp)
    for rk, tm in playoff_bracket.get_final().items():
        one_playoff_season.loc[
            (one_playoff_season["team_key"] == tm) & playoff_end_week_mask, "finish"
        ] = int(rk)

    if conso_teams:
        if len(conso_teams) < len(playoff_teams):
            conso_bracket = Tournament(conso_teams)
            num_of_rounds = int(math.ceil(math.log2(len(conso_teams))))
            for round_ in range(num_of_rounds):
                for match in conso_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = one_playoff_season["team_pts"][
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week + 1)
                    ].values[0]
                    right_pro_score = one_playoff_season["team_pro_pts"][
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week + 1)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = one_playoff_season["team_pts"][
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week + 1)
                    ].values[0]
                    left_pro_score = one_playoff_season["team_pro_pts"][
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week + 1)
                    ].values[0]
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp",
                    ] = left_comp
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp_pts",
                    ] = left_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp_pro_pts",
                    ] = left_pro_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp",
                    ] = right_comp
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp_pts",
                    ] = right_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp_pro_pts",
                    ] = right_pro_score
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        else:
            conso_bracket = Tournament(conso_teams)
            num_of_rounds = int(math.ceil(math.log2(len(conso_teams))))
            for round_ in range(num_of_rounds):
                for match in conso_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = one_playoff_season["team_pts"][
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week)
                    ].values[0]
                    right_pro_score = one_playoff_season["team_pro_pts"][
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = one_playoff_season["team_pts"][
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week)
                    ].values[0]
                    left_pro_score = one_playoff_season["team_pro_pts"][
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week)
                    ].values[0]
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp",
                    ] = left_comp
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp_pts",
                    ] = left_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp_pro_pts",
                    ] = left_pro_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp",
                    ] = right_comp
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp_pts",
                    ] = right_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp_pro_pts",
                    ] = right_pro_score
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        for rk, tm in conso_bracket.get_final().items():
            one_playoff_season.loc[
                (one_playoff_season["team_key"] == tm) & playoff_end_week_mask, "finish"
            ] = int(rk) + len(playoff_teams)

    if toilet_teams:
        if len(toilet_teams) < len(playoff_teams):
            toilet_bracket = Tournament(toilet_teams)
            num_of_rounds = int(math.ceil(math.log2(len(toilet_teams))))
            for round_ in range(num_of_rounds):
                for match in toilet_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = one_playoff_season["team_pts"][
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week + 1)
                    ].values[0]
                    right_pro_score = one_playoff_season["team_pro_pts"][
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week + 1)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = one_playoff_season["team_pts"][
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week + 1)
                    ].values[0]
                    left_pro_score = one_playoff_season["team_pro_pts"][
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week + 1)
                    ].values[0]
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp",
                    ] = left_comp
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp_pts",
                    ] = left_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp_pro_pts",
                    ] = left_pro_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp",
                    ] = right_comp
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp_pts",
                    ] = right_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week + 1),
                        "po_opp_pro_pts",
                    ] = right_pro_score
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        else:
            toilet_bracket = Tournament(toilet_teams)
            num_of_rounds = int(math.ceil(math.log2(len(toilet_teams))))
            for round_ in range(num_of_rounds):
                for match in toilet_bracket.get_active_matches():
                    right_comp = match.get_participants()[0].get_competitor()
                    right_score = one_playoff_season["team_pts"][
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week)
                    ].values[0]
                    right_pro_score = one_playoff_season["team_pro_pts"][
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week)
                    ].values[0]
                    left_comp = match.get_participants()[1].get_competitor()
                    left_score = one_playoff_season["team_pts"][
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week)
                    ].values[0]
                    left_pro_score = one_playoff_season["team_pro_pts"][
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week)
                    ].values[0]
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp",
                    ] = left_comp
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp_pts",
                    ] = left_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == right_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp_pro_pts",
                    ] = left_pro_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp",
                    ] = right_comp
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp_pts",
                    ] = right_score
                    one_playoff_season.loc[
                        (one_playoff_season["team_key"] == left_comp)
                        & (one_playoff_season["week"] == week),
                        "po_opp_pro_pts",
                    ] = right_pro_score
                    if right_score > left_score:
                        match.set_winner(right_comp)
                    elif right_score < left_score:
                        match.set_winner(left_comp)
                    elif right_score == left_score:
                        match.set_winner(left_comp)

        for rk, tm in toilet_bracket.get_final().items():
            one_playoff_season.loc[
                (one_playoff_season["team_key"] == tm) & playoff_end_week_mask, "finish"
            ] = (
                int(rk) + len(playoff_teams) + (len(conso_teams) if conso_teams else 0)
            )

    one_playoff_season.loc[playoff_end_week_mask, "finish"] = one_playoff_season.loc[
        playoff_end_week_mask, "finish"
    ].fillna(one_playoff_season["reg_season_rank"])

    one_playoff_season.sort_values(["week", "finish"], inplace=True)

    DatabaseCursor(PATH, option_schema="dev").copy_table_to_postgres_new(
        df=one_playoff_season, table=f"playoff_board_{str(key)}", first_time="YES"
    )

In [14]:
test = DatabaseCursor(PATH, option_schema="dev").copy_data_from_postgres(
    "SELECT * FROM prod.playoff_board_406"
)

Successfully pulled: SELECT * FROM prod.playoff_board_406


In [15]:
test

,game_id,finish,team_key,team_name,team_nickname,week,team_pts,team_pro_pts,ttl_pts_for_run,ttl_pro_pts_for_run,reg_season_rank,po_opp,po_opp_pts,po_opp_pro_pts
0,406,NaN,406.l.103661.t.1,Fantasy Sidelines,Tim,16,114.98,103.87,114.98,103.87,1,406.l.103661.t.2,88.50,128.69
1,406,NaN,406.l.103661.t.4,King of the Hill,Ryan,16,112.92,132.40,112.92,132.40,2,406.l.103661.t.7,125.00,123.71
2,406,NaN,406.l.103661.t.7,Harris & Harris RBs at Law 👨🏾‍⚖️,Greg,16,125.00,123.71,125.00,123.71,3,406.l.103661.t.4,112.92,132.40
3,406,NaN,406.l.103661.t.2,Dak to the Future,Wes,16,88.50,128.69,88.50,128.69,4,406.l.103661.t.1,114.98,103.87
4,406,NaN,406.l.103661.t.6,Wide Out University,Chris,16,83.86,80.28,83.86,80.28,5,NaN,NaN,NaN
5,406,NaN,406.l.103661.t.3,Fightin’ Blue Cock/Chubb/Wood,Kevin,16,156.88,110.97,156.88,110.97,6,NaN,NaN,NaN
6,406,NaN,406.l.103661.t.8,Poor Decisions 😢,Pete,16,94.38,109.24,94.38,109.24,7,406.l.103661.t.5,72.34,103.88
7,406,NaN,406.l.103661.t.9,Push Ups in Prison,Pat,16,124.54,117.68,124.54,117.68,8,406.l.103661.t.10,100.70,103.60
8,406,NaN,406.l.103661.t.10,Team 10,James,16,100.70,103.60,100.70,103.60,9,406.l.103661.t.9,124.54,117.68
9,406,NaN,406.l.103661.t.5,The Titanic,Jeremy,16,72.34,103.88,72.34,103.88,10,406.l.103661.t.8,94.38,109.24


MOM FFBL Draft Analysis/Trends Replacement

In [ ]:
"""
MoM FFBL Draft Analysis/Trends Replacement
"""
draft_query = """SELECT * FROM dev.draftresults"""
teams_query = """SELECT team_key
, game_id
, league_id
, "team_standings.outcome_totals.wins"
, "team_standings.outcome_totals.losses"
, number_of_moves
, number_of_trades
, nickname
, name
, draft_grade
, clinched_playoffs 
, "team_standings.playoff_seed"
, "team_standings.points_against" 
, "team_standings.points_for"
, "team_standings.rank"
FROM dev.leagueteams"""
players_query = """SELECT player_key
, "name.full"
, league_id
, game_id
, primary_position
, "bye_weeks.week"
, "draft_analysis.average_pick"
, "draft_analysis.average_round"
, "draft_analysis.percent_drafted"
 FROM dev.playerlist"""
draft = DatabaseCursor(PATH, option_schema="raw").copy_data_from_postgres(draft_query)
teams = DatabaseCursor(PATH, option_schema="raw").copy_data_from_postgres(teams_query)
players = DatabaseCursor(PATH, option_schema="raw").copy_data_from_postgres(
    players_query
)
draft_analysis = draft.merge(
    teams,
    how="left",
    left_on=["team_key", "game_id", "league_id"],
    right_on=["team_key", "game_id", "league_id"],
    suffixes=("_draft", "_teams"),
)
draft_analysis = draft_analysis.merge(
    players,
    how="left",
    left_on=["player_key", "game_id", "league_id"],
    right_on=["player_key", "game_id", "league_id"],
    suffixes=("", "_players"),
)
draft_analysis = draft_analysis[~draft_analysis["name"].isna()]
draft_analysis["clinched_playoffs"].fillna(0, inplace=True)
draft_analysis["team_standings.playoff_seed"].fillna(0, inplace=True)
encoded_grades = {
    "A+": 4,
    "A": 4,
    "A-": 3.7,
    "B+": 3.3,
    "B": 3.0,
    "B-": 2.7,
    "C+": 2.3,
    "C": 2.0,
    "C-": 1.7,
    "D+": 1.3,
    "D": 1.0,
    "na": np.nan,
}
draft_analysis["draft_gpa"] = draft_analysis["draft_grade"].replace(encoded_grades)
draft_analysis["avg_draft_gpa_rank"] = draft_analysis.groupby(["team_standings.rank"])[
    "draft_gpa"
].transform("mean")
draft_analysis["avg_draft_gpa_nickname"] = draft_analysis.groupby(["nickname"])[
    "draft_gpa"
].transform("mean")
draft_analysis["avg_draft_gpa_clinched_playoffs"] = draft_analysis.groupby(
    ["clinched_playoffs"]
)["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_playoffseed"] = draft_analysis.groupby(
    ["team_standings.playoff_seed"]
)["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_draft_pos"] = draft_analysis.groupby(["pick"])[
    "draft_gpa"
].transform("mean")

MoM FFBL League History Replacement

In [ ]:
"""
MoM FFBL League History Replacement
"""
matchups_query = """SELECT * FROM dev.weeklyleaguematchups"""
teams_query = """SELECT * FROM dev.leagueteams"""
draft_query = """SELECT * FROM dev.draftresults"""
players_query = """SELECT * FROM dev.playerlist"""
matchups = DatabaseCursor(PATH, option_schema="dev").copy_data_from_postgres(
    matchups_query
)
teams = DatabaseCursor(PATH, option_schema="dev").copy_data_from_postgres(teams_query)
draft = DatabaseCursor(PATH, option_schema="dev").copy_data_from_postgres(draft_query)
players = DatabaseCursor(PATH, option_schema="dev").copy_data_from_postgres(
    players_query
)
matchups = matchups.merge(
    teams,
    how="left",
    left_on=["game_id", "league_id", "team_a_team_key"],
    right_on=["game_id", "league_id", "team_key"],
    suffixes=("", "_team_a"),
)
matchups = matchups.merge(
    teams,
    how="left",
    left_on=["game_id", "league_id", "team_b_team_key"],
    right_on=["game_id", "league_id", "team_key"],
    suffixes=("_team_a", "_team_b"),
)

matchups[
    (matchups["is_playoffs"] == 1) & (matchups["is_consolation"] == 0)
].sort_values(
    ["week_start", "is_consolation", "winner_team_key"], ascending=[False, True, True]
)

MoM FFBL League Records Replacement

In [ ]:
"""
MoM FFBL League Records Replacement
"""